<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/gdocs_rag_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building RAG Chatbot with LangChain

In [21]:
!pip install -qU \
    openai \
    langchain \
    tiktoken \
    opensearch-py \
    sentence-transformers
#     langchain_community \
#     langchain_text_splitters


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.1 MB/s eta 0:00:00


In [8]:
# import os
# from langchain.chat_models import ChatOpenAI
# from google.colab import userdata

# OS_TOKEN = userdata.get('OS_TOKEN')
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# os.environ["OS_TOKEN"] = userdata.get('OS_TOKEN')
# chat = ChatOpenAI(
#     openai_api_key=os.environ["OPENAI_API_KEY"],
#     model='gpt-3.5-turbo'
# )

## setup generic OS gpy.docs index

In [10]:
# from langchain_community.document_loaders import TextLoader
# from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_community.embeddings import OpenAIEmbeddings
# from langchain_text_splitters import CharacterTextSplitter

In [11]:
import os
# from langchain.chat_models import ChatOpenAI
from google.colab import userdata

OS_TOKEN = userdata.get('OS_TOKEN')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OS_TOKEN"] = userdata.get('OS_TOKEN')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
# chat = ChatOpenAI(
#     openai_api_key=os.environ["OPENAI_API_KEY"],
#     model='gpt-3.5-turbo'
# )

In [15]:
# from langchain.embeddings import HuggingFaceEmbeddings

# init open-source embedding model (downloaded from hugging-face hub)
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
embeddings = OpenAIEmbeddings()
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
# embeddings

In [16]:
from langchain.vectorstores import OpenSearchVectorSearch

# Init OpenSearch client connection
docsearch = OpenSearchVectorSearch(
     index_name="pygraphistry-docs",  # TODO: use the same index-name used in the ingestion script
     embedding_function=embeddings,
     opensearch_url=OS_TOKEN,  # TODO: e.g. use the AWS OpenSearch domain instantiated previously
#     http_auth=("<<insert_user_name>>", "<<insert_password>>"),
#     use_ssl = False,
#     verify_certs = False,
#     ssl_assert_hostname = False,
#     ssl_show_warn = False,
)


In [17]:
docsearch.engine

In [23]:
query = "How can i use gfql chain?"
docs = docsearch.similarity_search(query, k=10)
print(docs[0].page_content)

See help(g.dbscan) or help(g.transform_dbscan) for options
Quickly configurable
Set visual attributes through quick data bindings and set all sorts of URL options. Check out the tutorials on colors, sizes, icons, badges, weighted clustering and sharing controls:
  g
    .privacy(mode='private', invited_users=[{'email': 'friend1@site.ngo', 'action': '10'}], notify=False)
    .edges(df, 'col_a', 'col_b')
    .edges(my_transform1(g._edges))
    .nodes(df, 'col_c')
    .nodes(my_transform2(g._nodes))
    .bind(source='col_a', destination='col_b', node='col_c')
    .bind(
      point_color='col_a',
      point_size='col_b',
      point_title='col_c',
      point_x='col_d',
      point_y='col_e')
    .bind(
      edge_color='col_m',
      edge_weight='col_n',
      edge_title='col_o')
    .encode_edge_color('timestamp', ["blue", "yellow", "red"], as_continuous=True)
    .encode_point_icon('device_type', categorical_mapping={'macbook': 'laptop', ...})


In [24]:
docsearch = OpenSearchVectorSearch.from_documents(
    docs,
    embeddings,
    index_name="pygraphistry-docs",  # TODO: use the same index-name used in the ingestion script
    # embedding_function=embeddings,
    opensearch_url=OS_TOKEN,
    engine="faiss",
    space_type="innerproduct",
    ef_construction=256,
    m=48,
)

# query = "how"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

See help(g.dbscan) or help(g.transform_dbscan) for options
Quickly configurable
Set visual attributes through quick data bindings and set all sorts of URL options. Check out the tutorials on colors, sizes, icons, badges, weighted clustering and sharing controls:
  g
    .privacy(mode='private', invited_users=[{'email': 'friend1@site.ngo', 'action': '10'}], notify=False)
    .edges(df, 'col_a', 'col_b')
    .edges(my_transform1(g._edges))
    .nodes(df, 'col_c')
    .nodes(my_transform2(g._nodes))
    .bind(source='col_a', destination='col_b', node='col_c')
    .bind(
      point_color='col_a',
      point_size='col_b',
      point_title='col_c',
      point_x='col_d',
      point_y='col_e')
    .bind(
      edge_color='col_m',
      edge_weight='col_n',
      edge_title='col_o')
    .encode_edge_color('timestamp', ["blue", "yellow", "red"], as_continuous=True)
    .encode_point_icon('device_type', categorical_mapping={'macbook': 'laptop', ...})


# sprinkle in OpenAI magic

In [25]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

In [26]:
from langchain.chat_models import ChatOpenAI


In [27]:
chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [28]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = docsearch.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [29]:
prompt = HumanMessage(
    content=augment_prompt(
        "how can i use UMAP in pygraphistry?"
    )
)

res = chat(messages + [prompt])
print(res.content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


To use UMAP in PyGraphistry, you can install PyGraphistry with the optional graphistry[ai] dependencies. This will add support for UMAP along with other features like graph autoML and graph neural networks. By combining PyGraphistry with these dependencies, you can easily utilize UMAP for your graph analysis and visualization tasks.


In [30]:
prompt = HumanMessage(
    content=augment_prompt(
        "how can i run gfql in pygraphistry?"
    )
)

res = chat(messages + [prompt])
print(res.content)

To run GFQL (Graphistry Frontend Query Language) in PyGraphistry, you can start by creating a graphistry client object and then use the `gfql` method to execute your GFQL queries. Here's a simple example:

```python
import graphistry

# Create a PyGraphistry client
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="your_username", password="your_password")

# Define your GFQL query
query = "YOUR_GFQL_QUERY_HERE"

# Execute the GFQL query
result = graphistry.gfql(query)

# Display or process the result as needed
print(result)
```

Make sure to replace `"your_username"` and `"your_password"` with your actual credentials and `"YOUR_GFQL_QUERY_HERE"` with your specific GFQL query. This code snippet demonstrates how you can run GFQL queries in PyGraphistry.


In [31]:
prompt = HumanMessage(
    content=augment_prompt(
        "will that run easily?"
    )
)

res = chat(messages + [prompt])
print(res.content)

Based on the provided context, it seems like the query is related to the ease of running PyGraphistry for exploring relationships. Yes, PyGraphistry is designed to be easy to install and use. You can install the client in your notebook or web app using pip, connect to a free Graphistry Hub account, or launch your own private GPU server. The installation process is straightforward, and you can choose different data plugins or AI modules based on your requirements.
